In [1]:
## Imports

import csv
import pandas as pd 
import time

In [3]:
# Setup Twilio interface
import os
from twilio.rest import Client

account_sid = os.environ['TWILIO_ACCOUNT_SID'] = "AC9077560e954b4c3f7d41e071d7ce11fa"
auth_token = os.environ['TWILIO_AUTH_TOKEN'] = "29c0fede2bae7fb2f0233010be12ece6"
client = Client(account_sid, auth_token)

In [4]:
## Setup

# Paths
PATH = 'C:/Users/Sirpimmon/Desktop/hackathon22s/plantbuddy/'
conditions_path = "outputs/conditions.csv"
temps_path = "outputs/plant_temp_ranges.csv"
readings = "readings/reading1.csv" # For when it's using the renamed sheet
# readings = "readings/out.csv" # For when it's read directly from the arduino

# Time interval
unit = 0 # 0 for seconds, 1 for minutes
interval = 5 # 5 for minutes, 20 for seconds for demonstration

In [5]:
## Make dictionary of plants

filename = PATH + temps_path
file = open(filename)

csvreader = csv.reader(file)
plant_dict = {}

header = []
header = next(csvreader)

for row in csvreader:
    plant_dict[row[0]] = (float(row[2]), float(row[1]))

file.close()

In [6]:
# Read conditions csv

filename = PATH + conditions_path
conditions = pd.read_csv(filename, delimiter = ',', encoding="utf-8-sig")

In [7]:
# Ideal plant conditions from file

max_press = conditions._get_value(0, "max_press")
min_press = conditions._get_value(0, "min_press")
max_humid = conditions._get_value(0, "max_humid")
min_humid = conditions._get_value(0, "min_humid")
low_light_threshold = conditions._get_value(0, "low_light_threshold")
high_light_threshold = conditions._get_value(0, "high_light_threshold")

In [8]:
## Take input

print("What type of plant do you have?")
x = input().lower()

# If it's not in the American plants list, then it's assumed it's a houseplant
if x not in plant_dict:
    plant_dict[x] = (60, 80)

What type of plant do you have?
iris


In [9]:
## Read values from arduino
def readvals(filename):
    global light
    global temp
    global time
    global humidity
    global pressure
    
    
    file = open(filename)

    csvreader = csv.reader(file)

    header = []
    header = next(csvreader)

    for row in csvreader:
        if unit == 0:
            time = int(row[0][3:5])
        else:
            time = int(row[0][0:2])
        light = int(row[1])
        temp = (float(row[2])*1.8) + 32
        humidity = float(row[3])
        pressure = float(row[4])

    file.close()


In [16]:
# Loop until stopped
while True:
    
    ## Reports
    prev_time_recorded = 0

    # If it's time to report a status update
    if time%interval == 0 and time != prev_time_recorded:
        prev_time_recorded = time
            
        # Read values from Arduino
        filename = PATH + readings    
        readvals(filename)


        ## Check levels - 0 for good level, 1 for too low, 2 for too high
        light_lvl = 0
        temp_lvl = 0
        humidity_lvl = 0
        pressure_lvl = 0

        if light < low_light_threshold:
            light_lvl = 1
        elif light > high_light_threshold:
            light_lvl = 2

        if temp < plant_dict[x][0]:
            temp_lvl = 1
        elif temp > plant_dict[x][1]:
            temp_lvl = 2

        if humidity < min_humid:
            humidity_lvl = 1
        elif humidity > max_humid:
            humidity_lvl = 2
            
        if pressure < min_press:
            pressure_lvl = 1
        elif pressure > max_press:
            pressure_lvl = 2
            
        
        ## Msg generator
        msg = ""

        if light_lvl == 1:
            msg = msg + "Your plant needs more light, "
        elif light_lvl == 2:
            msg = msg + "Your plant needs less light, "

        if temp_lvl == 1:
            msg = msg + "to be warmed up to " + str(plant_dict[x][0]) + " degrees, "
        elif temp_lvl == 2:
            msg = msg + "to be cooled down to " + str(plant_dict[x][1]) + " degrees, "
            
        if humidity_lvl == 1:
            msg = msg + "and to be watered. "
        elif humidity_lvl == 2:
            msg = msg + "and to stop being watered. "
            
        if pressure_lvl == 1:
            msg = msg + "And more pressure."
        elif pressure_lvl == 2:
            msg = msg + "And less pressure."
            
        if temp_lvl + pressure_lvl + humidity_lvl + light_lvl == 0:
            msg = "Everything is great!"
        print(msg)
        
        message = client.messages \
            .create(
                body=msg,
                from_='+18438060190',
                to='+15018301647'
            )
        
        

Your plant needs more light, and to be watered. 
Your plant needs more light, and to be watered. 
Your plant needs more light, and to be watered. 
Your plant needs more light, and to be watered. 
Your plant needs more light, and to be watered. 
Your plant needs more light, and to be watered. 
Your plant needs more light, and to be watered. 
Your plant needs more light, and to be watered. 
Your plant needs more light, and to be watered. 
Your plant needs more light, and to be watered. 
Your plant needs more light, and to be watered. 
Your plant needs more light, and to be watered. 
Your plant needs more light, and to be watered. 
Your plant needs more light, and to be watered. 
Your plant needs more light, and to be watered. 
Your plant needs more light, and to be watered. 
Your plant needs more light, and to be watered. 
Your plant needs more light, and to be watered. 
Your plant needs more light, and to be watered. 
Your plant needs more light, and to be watered. 
Your plant needs mor

KeyboardInterrupt: 

In [12]:
# If the texting isn't implemented

msg = ""

if light_lvl == 1:
    print("I need more light. Please move me into the sun!")
elif light_lvl == 2:
    print("I need less light. Please move me into the shade!")
else:
    print("This much light is good!")

if temp_lvl == 1:
    print("I'm cold! Raise the temperature to " + str(plant_dict[x][0]) + " degrees.")
elif temp_lvl == 2:
    print("I'm hot! Lower the temperature to " + str(plant_dict[x][1]) + " degrees.")
else:
    print("This temperature is good!")
    
if humidity_lvl == 1:
    print("I'm thirsty! Please water me.")
elif humidity_lvl == 2:
    print("I'm too wet! Please stop watering me.")
else:
    print("This is a perfect amount of water!")
    
if pressure_lvl == 1:
    print("This isn't enough air pressure!")
elif pressure_lvl == 2:
    print("This is too much air pressure!")
else:
    print("This amount of pressure is good!")


I need more light. Please move me into the sun!
This temperature is good!
I'm thirsty! Please water me.
This amount of pressure is good!
